In [1]:
from unit_analysis import *


# #Richmond, VA
# fdid = '76000'
# state = 'VA'
# firecares_id = '93345'


fdid = '76000'
state = 'VA'
firecares_id = '93345'


rich = unit_analysis(firecares_id,fdid,state,load_all=True)
# rich.apparatus_query(load_dir='./apparatus_df')
# rich.first_due_analysis()
# rich.station_maker()

Acquiring deprartment's jurisdictional boundary...
Done! 



FileNotFoundError: [Errno 2] No such file or directory: './apparatus_df/93345apparatus'

In [ ]:
# rich.first_due_analysis()
rich.station_maker()

In [ ]:
dir(rich)

In [ ]:
from matplotlib import rcParams, cycler
plt.rcParams.update({'font.size': 12})
cmap = plt.cm.cool
plt.rcParams['figure.figsize'] = [20,20]
rcParams['axes.prop_cycle'] = plt.cycler(color=cmap(np.linspace(0, 1, 3)))




for i,station in enumerate(rich.station_objects):
    plt.subplot(5,5,i+1)

    try:
        x = station.sent_unit_times
        y =station.no_unit_not_busy
        z = station.no_unit_and_busy

        percentiles = np.linspace(0,100)
        sent_times = np.percentile(x,percentiles)
        no_unit_not_busy = np.percentile(y,percentiles)
        no_unit_busy = np.percentile(z,percentiles)



        l1 = plt.plot(sent_times, percentiles)
        l2 = plt.plot(no_unit_not_busy, percentiles)
        l3 = plt.plot(no_unit_busy, percentiles)
        plt.xlim([0,600])
        plt.xlabel('time (sec)')
        plt.ylabel('percentile')
        plt.title(str(i))
    except IndexError:
        pass
    
lines = [l1[0],l2[0],l3[0]]
legend_list = ['First due sends unit', 'First due does not send, but is not busy', 'First due does not send and is busy']
plt.figlegend( lines, legend_list, loc = 'upper center', ncol=1, labelspacing=0. ,prop={'size': 20})
# plt.legend(,loc=1)


In [ ]:
dir(rich.station_objects[1])

In [ ]:
times = np.percentile(x,percentiles)

In [ ]:
rich.station_objects[1].first_due_prob

In [ ]:
rich.first_due_plot()

In [ ]:
rich.station_plot()

In [ ]:
np.sum(rich.apparatus_df['unit_type']=='Engine')/len(rich.apparatus_df)

In [ ]:
np.unique(rich.apparatus_df['unit_type'])

In [ ]:
rich.time_simulator(num_iter=70000)

In [ ]:
#Modifying station reliability
for i,station_num in enumerate(rich.station_list):
    rich.station_objects[i].reliability['dispatch_prob'] = np.ones(len(rich.station_objects[i].reliability['dispatch_prob']))
rich.time_simulator(num_iter=70000)

In [ ]:
rich.station_objects[i].reliability['dispatch_prob'] = np.ones(len(rich.station_objects[i].reliability['dispatch_prob']))

In [ ]:
df = rich.first_due_df[(rich.first_due_df['first_due_busy']==True) & (rich.first_due_df['any_from_first_due']==False)  ].copy()
no_due_and_busy = heatmap(df,y_pred='travel_time',boundary=rich.boundary,cutoff_num=20,cutoff_rad=.5)

df = rich.first_due_df[(rich.first_due_df['any_from_first_due']==True)].copy()
first_due = heatmap(df,y_pred='travel_time',boundary=rich.boundary,cutoff_num=20,cutoff_rad=.5)

In [ ]:
dir(first_due)

In [ ]:
import numpy as np
import pandas as pd
import folium
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
import matplotlib as mpl




n=100
#Need to reshape data
y_grid = np.reshape(no_due_and_busy.y_pred-first_due.y_pred,[n,n]).T
vmin   = np.nanmin(y_grid)
vmax   = np.nanmax(y_grid)


# Setup colormap
levels = 20
cmap = mpl.cm.get_cmap('coolwarm')
colors = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0,1,levels)]
cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)


#Then making the contour plot
contourf = plt.contourf(first_due.longlocs,first_due.latlocs,y_grid,levels=levels,colors=colors,linestyles=None,vmin=vmin,vmax=vmax)







# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=0,
    ndigits=10,
    stroke_width=0,
    fill_opacity=0)

# Set up the folium plot
geomap = folium.Map([np.mean(first_due.latlocs), np.mean(first_due.longlocs)], zoom_start=10, tiles="cartodbpositron")

# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
#         'weight': .5,
#         'opacity': .5,
        'fillColor': x['properties']['fill'],
        'fillOpacity': .5
,
    }).add_to(geomap)

# Add the colormap to the folium map
cm.caption = 'Avg. delay due to first-due station being busy and unable to dispatch (sec)'
geomap.add_child(cm)


for i in rich.station_objects:
    try:
        folium.Marker(
            location = [ i.geom.coords[0][1],  i.geom.coords[0][0]],
            icon=folium.Icon(color='red',icon='fire'),
    #         tooltip=tooltip
        ).add_to(geomap)
    except(AttributeError):
        pass




# Fullscreen mode
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)


# Plot the contour plot on folium
folium.GeoJson(
    rich.boundary,
    style_function=lambda x: {
        'color':     'black',
        'weight':   5,
#         'fillColor': x['properties']['fill'],
        'fillOpacity': 0.0,
    }).add_to(geomap)




plt.close()
geomap



# Integration check
The area integral of the density should be close (but less than) the total number of incidents

In [ ]:
delta_x = xlocs[1] - xlocs[0]
delta_y = ylocs[1] - ylocs[0]
area = delta_x*delta_y
areas = np.ones(len(density_full))*area
np.nansum(areas*density_full)

In [ ]:
len(df)

In [ ]:
station_df.iloc[0]

In [ ]:
rich.station_list[0] == station_df.iloc[0]['station_number']

In [ ]:
station_df.iloc[np.where(rich.station_list == station_df.iloc[0]['station_number'])[0][0]]['geom']

In [ ]:
station_df.iloc[np.where(station_df['station_number'] == 100)[0][0]]['geom']

In [ ]:
import numpy as np
import pandas as pd
import folium
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
import matplotlib as mpl





#Need to reshape data
y_grid = np.reshape(test.y_pred,[n,n]).T
vmin   = np.nanmin(y_grid)
vmax   = np.nanmax(y_grid)


# Setup colormap
levels = 20
cmap = mpl.cm.get_cmap('coolwarm')
colors = [mpl.colors.rgb2hex(cmap(i)) for i in np.linspace(0,1,levels)]
cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)


#Then making the contour plot
contourf = plt.contourf(test.longlocs,test.latlocs,y_grid,levels=levels,colors=colors,linestyles=None,vmin=vmin,vmax=vmax)







# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=0,
    ndigits=10,
    stroke_width=0,
    fill_opacity=0)

# Set up the folium plot
geomap = folium.Map([np.mean(test.latlocs), np.mean(test.longlocs)], zoom_start=10, tiles="cartodbpositron")

# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
#         'weight': .5,
#         'opacity': .5,
        'fillColor': x['properties']['fill'],
        'fillOpacity': .5
,
    }).add_to(geomap)

# Add the colormap to the folium map
cm.caption = 'Avg. delay due to first-due station being busy and unable to dispatch (sec)'
geomap.add_child(cm)


for i in rich.station_objects:
    try:
        folium.Marker(
            location = [ i.geom.coords[0][1],  i.geom.coords[0][0]],
            icon=folium.Icon(color='red',icon='fire'),
    #         tooltip=tooltip
        ).add_to(geomap)
    except(AttributeError):
        pass




# Fullscreen mode
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)


# Plot the contour plot on folium
folium.GeoJson(
    rich.boundary,
    style_function=lambda x: {
        'color':     'black',
        'weight':   5,
#         'fillColor': x['properties']['fill'],
        'fillOpacity': 0.0,
    }).add_to(geomap)




plt.close()
geomap
